In [ ]:
from pathlib import Path
import sys

DATA_DIR = Path.home() / "projects" / "data"

csv_file = DATA_DIR / "raw" /"GSE290585_SeSaMeBeta_MM285_BS.csv" # path to mouse csv file

print(sys.executable)
print(csv_file.exists())

OUTPUT_DIR = DATA_DIR / "processed"

In [ ]:
import pandas as pd

df = pd.read_csv(csv_file)

with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df.head(20))


In [ ]:
# Function to simplify column names. It will look at a df.column from the back of the string and count n_underscores.
# It will remove everything to the right of this underscore (including that last underscore).
# If there are any duplicates labeled as with suffix".*any_digit*", these will be temporarily stored and conjencated back to the string.
# Some of the columns ended with a suffix "_ACEseq". These have been handled with explicit if conditioning.
import warnings

def simplify_colname(name: str, n_underscores: int = 4) -> str:
    """
    Simplify column names by counting 'n_underscores' from the back of the string
    and removing everything to the right of that underscore (including it).

    Any duplicate suffixes of the form ".<digit>" (such as ".1" or ".2") are temporarily stored and
    concatenated back to the simplified name.

    Some columns end with the suffix "_ACEseq"; these are handled explicitly
    with conditional logic to avoid breaking tissue names that contain
    underscores.

    A warning is raised if a column contains fewer than 'n_underscores'
    underscores.
    """

    #1. split on dots
    parts = name.split(".")

    #2. check for trailing numeric suffix
    if len(parts) > 1 and parts[-1].isdigit():
        suffix = "." + parts[-1]
        core = ".".join(parts[:-1])
    else:
        suffix = ""
        core = name


    #SPECIAL CASE: ACE-seq columns
    if core.endswith("_ACEseq"):
        core = core.replace("_ACEseq", "")

    #3. count underscores
    underscore_count = core.count("_")
    if underscore_count < n_underscores:
        warnings.warn(
            f"Column '{name}' has only {underscore_count} underscores (< {n_underscores}); leaving unchanged."
        )
        return name

    #4. split from the right and keep left part
    simplified = core.rsplit("_", n_underscores)[0]

    #5. add suffix back
    return simplified + suffix


In [ ]:
# Define a function to group same cell type together. Preferably with any duplicates going from "Tail, Tail.1, Tail.2".
# Later make a list on how many of each cell type (how many columns).

def reorder_columns_grouped(df):
    """
    Reorder columns by grouping base tissue/cell type names together and sorting
    numeric suffixes (.1, .2, .3, ...) within each group.

    Safe with duplicate column names because reordering is done by column position.
    Keeps the first column unchanged.
    """

    def col_sort_key(col: str):
        parts = col.split(".", 1)
        base = parts[0]
        num = int(parts[1]) if len(parts) == 2 and parts[1].isdigit() else 0
        return (base, num)

    # indices for all columns except the first
    other_idx = list(range(1, df.shape[1]))

    # sort indices using the column-name-based key
    other_idx_sorted = sorted(
        other_idx,
        key=lambda i: col_sort_key(df.columns[i])
    )

    # keep first column first
    new_idx_order = [0] + other_idx_sorted

    # reorder by position (moves full columns safely)
    return df.iloc[:, new_idx_order]


In [ ]:
# Test if working so far

tests = [
    "Blood_Male_36_weeks_BS",
    "Tail_Female_8_weeks_BS.2",
    "probe_ID"
]

for t in tests:
    print(t, "->", simplify_colname(t))


In [ ]:
# visually inspect the columns
original_columns = df.columns
original_columns[:10]

# we do not want to change the first column, so here we separate them
first_col = df.columns[0]
other_cols = df.columns[1:]

# loop through the columns
simplified_other_cols = []

for col in other_cols:
    new_col = simplify_colname(col, n_underscores=4)
    simplified_other_cols.append(new_col)


# Reconstruct the full column list with first column:
new_columns = [first_col] + simplified_other_cols


In [ ]:
# critical inspection step to see if it works
for old, new in zip(df.columns[-10:], new_columns[-10:]):
    print(f"{old}  ->  {new}")    

In [ ]:
# assign new column names to the df
df.columns = new_columns

# visual inspection
df.columns[:10]
df.columns.is_unique # will return false due to current design choices

# Call re-order function:

df_reordered = reorder_columns_grouped(df)



In [ ]:
#Before reordering
with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df.head(20))

before_reordering = (df.columns == "Adrenal.1").sum()
print("before: ")
print(before_reordering)



In [ ]:
#After reordering
with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df_reordered.head(20))


after_reordering = (df_reordered.columns == "Adrenal.1").sum()
print("after: ")
print(after_reordering)


In [ ]:
#Count how many columns in each cell type group
cell_type_counts = df_reordered.columns[1:].str.split(".", n=1).str[0].value_counts()
print(cell_type_counts)

cell_type_counts.unique()


In [ ]:
output_path = OUTPUT_DIR / (csv_file.stem + "_simplified_columns.csv")

df.to_csv(output_path, index=False)

print(f"Saved to: {output_path}")